In [23]:
import yfinance as yf
import pandas as pd

def get_1d_stock_data(ticker_symbol, period="1y", filename=None):
    """
    Fetch historical stock data with daily intervals, add VWAP, and optionally save to CSV or Excel.

    Args:
        ticker_symbol (str): Stock ticker symbol (e.g., "AAPL", "GOOG").
        period (str): Period of data (e.g., "1d", "5d", "30d").
        filename (str): If set, saves the data to this file (.csv or .xlsx).

    Returns:
        pd.DataFrame: Historical stock data with daily intervals and VWAP.
    """
    try:
        # Fetch stock data
        stock = yf.Ticker(ticker_symbol)
        data = stock.history(interval="1d", period=period)

        # Debug: Print the columns and first few rows to check the structure
        print("Columns before reset:", data.columns)
        print("First few rows before reset:\n", data.head())

        # Reset index so 'Date' becomes a column
        data.reset_index(inplace=True)

        # Debug: Print the columns and first few rows after reset
        print("Columns after reset:", data.columns)
        print("First few rows after reset:\n", data.head())

        # If 'Date' column exists, remove timezone info for Excel compatibility
        if 'Date' in data.columns:
            data['Date'] = data['Date'].dt.tz_localize(None)

        # Calculate typical price (average of High, Low, and Close)
        data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3

        # VWAP calculation per day
        data['TPV'] = data['Typical_Price'] * data['Volume']
        data['Cum_TPV'] = data['TPV'].cumsum()  # Cumulative sum over the period
        data['Cum_Volume'] = data['Volume'].cumsum()  # Cumulative sum of volume
        data['VWAP'] = data['Cum_TPV'] / data['Cum_Volume']

        # Drop intermediate columns if you want
        data.drop(columns=['TPV', 'Cum_TPV', 'Cum_Volume', 'Typical_Price'], inplace=True)

        # Save the data to a CSV or Excel file
        if filename:
            if not filename.endswith((".csv", ".xlsx")):
                filename += ".csv"  # Default to CSV if extension is not specified
            if filename.endswith(".csv"):
                data.to_csv(filename, index=False)
                print(f"Data saved to {filename}")
            else:
                data.to_excel(filename, index=False)
                print(f"Data saved to {filename}")

        return data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Example usage
ticker = "NVDA"
df = get_1d_stock_data(ticker, period="1y", filename="NVDA_1d_1y.csv")
if df is not None:
    # Checking the columns before attempting to access the 'Date' column
    print(df.columns)

    # Make sure 'Date' is in the columns
    if 'Date' in df.columns:
        print(df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
    else:
        print("The 'Date' column is not present!")

Columns before reset: Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')
First few rows before reset:
                                 Open       High        Low      Close  \
Date                                                                    
2024-04-12 00:00:00-04:00  89.669539  90.145389  87.501253  88.157036   
2024-04-15 00:00:00-04:00  89.068723  90.583224  85.900765  85.972740   
2024-04-16 00:00:00-04:00  86.404604  88.089048  86.035730  87.386284   
2024-04-17 00:00:00-04:00  88.310971  88.745833  83.922414  84.007393   
2024-04-18 00:00:00-04:00  84.942086  86.161687  82.374929  84.643181   

                              Volume  Dividends  Stock Splits  
Date                                                           
2024-04-12 00:00:00-04:00  426805000        0.0           0.0  
2024-04-15 00:00:00-04:00  443077000        0.0           0.0  
2024-04-16 00:00:00-04:00  370453000        0.0           0.0  
2024-04-17 00:00:00-0